In [1]:
#importing libraries
import numpy as np
import pandas as pd
import re
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('mdc-1.csv')
data.tail(10) #display last 10 rows in data

,id,activity_id,name
3426,"3,192,802",MDC1-10606,COLO-2 Cell-4 UPS PNL01 Compass issues Prevent...
3427,"3,194,335",MDC1.C1.Cx3020.1,Additional Go-Back loadbanking (ASCO)
3428,"3,192,808",MDC1-10607,6.1-98 Admin/COLO-1 ASCO EPMS Connectivity Issues
3429,"3,192,809",MDC1-10608,6.1-88 Admin/ COLO-1 MER/ADMIN UPS L3 Testing...
3430,"3,192,810",MDC1-10609,6.1-106 COLO-1 Cell-2 UPS01 Power module failure
3431,"3,192,812",MDC1-10610,COLO-2 Cell-1 Open Issues preventing Yellow ta...
3432,"3,192,818",MDC1-10611,COLO-2 Cell-2 Open Issues preventing Yellow ta...
3433,"3,192,813",MDC1-10612,COLO-2 Cell-3 Open Issues preventing Yellow ta...
3434,"3,192,814",MDC1-10613,COLO-2 Cell-4 Open Issues preventing Yellow ta...
3435,"3,192,816",MDC1-10614,COLO-3 Delay in SE uploading QC Checkilst


In [42]:
### Divide data set into 1-word names and multiword names
### All 1-word rows are clusters.

In [4]:
data['lower_case_names'] = data['name'].str.lower() #change make column a lowercase

In [5]:
data.head()

,id,activity_id,name,lower_case_names
0,"3,191,380",MDC1-UP#13,MDC1 Master Update #13- DD 8.31.2021Submitted,mdc1 master update #13- dd 8.31.2021submitted
1,"3,191,381",MDC1-UP#13.1,Executive Summary / Milestones,executive summary / milestones
2,"3,192,817",MDC1-UP#13.5,Construction,construction
3,"3,191,388",MDC1-UP#13.2,Construction Summary,construction summary
4,"3,191,673",MDC1-UP#13.4,Preconstruction,preconstruction


In [43]:
##check for 1-word in table

In [6]:
data['is_one_word_name'] = data['lower_case_names'].map(lambda n: len(n.split(" ")) == 1)

In [7]:
data.head()

,id,activity_id,name,lower_case_names,is_one_word_name
0,"3,191,380",MDC1-UP#13,MDC1 Master Update #13- DD 8.31.2021Submitted,mdc1 master update #13- dd 8.31.2021submitted,False
1,"3,191,381",MDC1-UP#13.1,Executive Summary / Milestones,executive summary / milestones,False
2,"3,192,817",MDC1-UP#13.5,Construction,construction,True
3,"3,191,388",MDC1-UP#13.2,Construction Summary,construction summary,False
4,"3,191,673",MDC1-UP#13.4,Preconstruction,preconstruction,True


In [8]:
data_1_word = data[data.is_one_word_name == True]

In [9]:
data_1_word.head()

,id,activity_id,name,lower_case_names,is_one_word_name
2,"3,192,817",MDC1-UP#13.5,Construction,construction,True
4,"3,191,673",MDC1-UP#13.4,Preconstruction,preconstruction,True
6,"3,192,002",MDC1-UP#13.6,DELAY,delay,True
9,"3,192,819",MDC1-UP#13.5.1,Site,site,True
13,"3,191,600",MDC1-UP#13.3.1,General,general,True


In [10]:
data_1_word.shape #shape of 1-word

(419, 5)

In [11]:
data_multiword = data[data.is_one_word_name == False]

In [12]:
data_multiword.shape #shape of multiple words

(3017, 5)

In [13]:
data.shape 

(3436, 5)

In [14]:
419/3436 #checking ratio of 1-word to multiple words

0.12194412107101281

In [15]:
### Second Stage
##  Create a dictionary of all one-word names mapped to a list
##  Iterate over each

In [16]:
def make_cluser_dictionary(df):
    clusters = dict()
    for i, row in df.iterrows():
        name = row['lower_case_names']
        clusters[name] = []
    return clusters

In [17]:
cl = make_cluser_dictionary(data_1_word)

In [18]:
cl.keys()

dict_keys(['construction', 'preconstruction', 'delay', 'site', 'general', 'admin/colo1', 'fitout', 'commissioning', 'colo-2', 'colo-3', 'colo-4', 'update#4', 'punchlist', 'update#6', 'update#7', 'update#8', 'update#10', 'update#12', 'admin', 'colo-1', 'admin/colo-1', 'structure', 'l2', 'l3', 'mer', 'drb#1', 'term', 'ahu', 'upm', 'generator', 'pdu', 'procurement', 'se', 'foundations', 'roof', 'partitions', 'finishes', 'specialities', 'electrical', 'plumbing', 'mechanical', 'vaults', 'cell-1', 'cell-3', 'cell-2', 'cell-4', 'jaz', 'fiber', 'building', 'west', 'east', 'mobilization', '`', 'rfi#432'])

In [46]:
cl  #display 1-word dictionaries

{'construction': [],
 'preconstruction': [],
 'delay': [],
 'site': [],
 'general': [],
 'admin/colo1': [],
 'fitout': [],
 'commissioning': [],
 'colo-2': [],
 'colo-3': [],
 'colo-4': [],
 'update#4': [],
 'punchlist': [],
 'update#6': [],
 'update#7': [],
 'update#8': [],
 'update#10': [],
 'update#12': [],
 'admin': [],
 'colo-1': [],
 'admin/colo-1': [],
 'structure': [],
 'l2': [],
 'l3': [],
 'mer': [],
 'drb#1': [],
 'term': [],
 'ahu': [],
 'upm': [],
 'generator': [],
 'pdu': [],
 'procurement': [],
 'se': [],
 'foundations': [],
 'roof': [],
 'partitions': [],
 'finishes': [],
 'specialities': [],
 'electrical': [],
 'plumbing': [],
 'mechanical': [],
 'vaults': [],
 'cell-1': [],
 'cell-3': [],
 'cell-2': [],
 'cell-4': [],
 'jaz': [],
 'fiber': [],
 'building': [],
 'west': [],
 'east': [],
 'mobilization': [],
 '`': [],
 'rfi#432': []}

In [23]:
#compares key and extracts multiwords as values
def build_clusters(df_source, df_clusters):
    clusters = make_cluser_dictionary(df_clusters)
    for cluster_key in clusters.keys():
        for i,j in df_source.iterrows():
#             print(i,j['lower_case_names'])
            name = j['lower_case_names']
            
            if cluster_key in name:
                clusters[cluster_key].append(name)
        ## iterate over df_source rows
    return clusters

In [24]:
name_clusters = build_clusters(data_multiword, data_1_word)

In [25]:
name_clusters 

{'construction': ['construction summary',
  'bas prefunctional testing on construction server',
  'bas prefunctional testing on construction server',
  'bas prefunctional testing on construction server',
  'bas prefunctional testing on construction server',
  'jaz riser support drb#1 procurement/ jaz foundations/construction',
  'construction server- colo-1',
  'construction server- colo-2',
  'construction server- colo-3',
  'construction server- colo-4',
  'all construction activities -safety standown due to admin gen unplanned energization'],
 'preconstruction': [],
 'delay': ['executed delays (co-66)',
  'pending delays',
  'delay #1 late submittal approval',
  'delay #3 fluid cooler',
  'delay #2 late upm deliveries',
  'delay #4 drb#2',
  'delay #5 water treatment skids delay',
  'delay #6 drb#3',
  'delay #7- force majeure shutdown -hazardous air quality index',
  'delay #8-underground utilities impacted by overhead power lines',
  'delay #9 mer delivery delay',
  'delay#10 covi

In [27]:
#stores clusters in json file
import json
with open('cluster.json', 'w') as f:
    json.dump(name_clusters, f)

In [28]:
name_clusters['construction']

['construction summary',
 'bas prefunctional testing on construction server',
 'bas prefunctional testing on construction server',
 'bas prefunctional testing on construction server',
 'bas prefunctional testing on construction server',
 'jaz riser support drb#1 procurement/ jaz foundations/construction',
 'construction server- colo-1',
 'construction server- colo-2',
 'construction server- colo-3',
 'construction server- colo-4',
 'all construction activities -safety standown due to admin gen unplanned energization']

In [29]:
## FINAL STEP - Add to original
items = { 1:2 }
1 in items

True

In [30]:
def map_cluster_to_cluster_name(name_clusters):
    index = 1;
    output = dict()
    for key in name_clusters.keys():
        values = name_clusters[key]
        values.append(key)
        prefix = "Cluster#{}"
        cluster_name = prefix.format(index)

        for v in values:
            output[v] = cluster_name
        index += 1
    print(index)
    return output
            

In [31]:
map_cluster_to_cluster_name(name_clusters)

55


{'construction summary': 'Cluster#1',
 'bas prefunctional testing on construction server': 'Cluster#33',
 'jaz riser support drb#1 procurement/ jaz foundations/construction': 'Cluster#47',
 'construction server- colo-1': 'Cluster#33',
 'construction server- colo-2': 'Cluster#33',
 'construction server- colo-3': 'Cluster#33',
 'construction server- colo-4': 'Cluster#33',
 'all construction activities -safety standown due to admin gen unplanned energization': 'Cluster#19',
 'construction': 'Cluster#1',
 'preconstruction': 'Cluster#2',
 'executed delays (co-66)': 'Cluster#3',
 'pending delays': 'Cluster#3',
 'delay #1 late submittal approval': 'Cluster#3',
 'delay #3 fluid cooler': 'Cluster#3',
 'delay #2 late upm deliveries': 'Cluster#29',
 'delay #4 drb#2': 'Cluster#3',
 'delay #5 water treatment skids delay': 'Cluster#3',
 'delay #6 drb#3': 'Cluster#3',
 'delay #7- force majeure shutdown -hazardous air quality index': 'Cluster#3',
 'delay #8-underground utilities impacted by overhead p

In [32]:
data["cluster"] = data['lower_case_names'].map(map_cluster_to_cluster_name(name_clusters))

55


In [33]:
## proper mapping

def map_cluster_name(lower_case_name):
    mapping = map_cluster_to_cluster_name(name_clusters)
    if lower_case_name in mapping:
        return mapping[lower_case_name]
    else:
        return np.nan

In [34]:
data["cluster"] = data['lower_case_names'].map(map_cluster_name)

55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
5

55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
5

In [35]:
data.head()

,id,activity_id,name,lower_case_names,is_one_word_name,cluster
0,"3,191,380",MDC1-UP#13,MDC1 Master Update #13- DD 8.31.2021Submitted,mdc1 master update #13- dd 8.31.2021submitted,False,NaN
1,"3,191,381",MDC1-UP#13.1,Executive Summary / Milestones,executive summary / milestones,False,NaN
2,"3,192,817",MDC1-UP#13.5,Construction,construction,True,Cluster#1
3,"3,191,388",MDC1-UP#13.2,Construction Summary,construction summary,False,Cluster#1
4,"3,191,673",MDC1-UP#13.4,Preconstruction,preconstruction,True,Cluster#2


In [36]:
data.tail(60)

,id,activity_id,name,lower_case_names,is_one_word_name,cluster
3376,"3,192,751",MDC1-10554,COLO-4 Cell-2 MER-UPS Battery Disconnect modif...,colo-4 cell-2 mer-ups battery disconnect modif...,False,Cluster#45
3377,"3,192,752",MDC1-10555,COLO-4 Cell-3 MER-UPS Battery Disconnect modif...,colo-4 cell-3 mer-ups battery disconnect modif...,False,Cluster#44
3378,"3,192,753",MDC1-10556,COLO-4 Cell-4 MER-UPS Battery Disconnect modif...,colo-4 cell-4 mer-ups battery disconnect modif...,False,Cluster#46
3379,"3,192,758",MDC1-10557,COLO-3 Cell-4 Remove inaccessible tray in MER,colo-3 cell-4 remove inaccessible tray in mer,False,Cluster#46
3380,"3,192,759",MDC1-10558,COLO-3 Cell-4 Reinstall tray in MER,colo-3 cell-4 reinstall tray in mer,False,Cluster#46
3381,"3,192,760",MDC1-10560,COLO-4 Cell-1 Remove inaccessible tray in MER,colo-4 cell-1 remove inaccessible tray in mer,False,Cluster#43
3382,"3,192,761",MDC1-10561,COLO-4 Cell-1 Reinstall tray in MER,colo-4 cell-1 reinstall tray in mer,False,Cluster#43
3383,"3,192,762",MDC1-10562,COLO-4 Cell-2 Remove inaccessible tray in MER,colo-4 cell-2 remove inaccessible tray in mer,False,Cluster#45
3384,"3,192,763",MDC1-10563,COLO-4 Cell-2 Reinstall tray in MER,colo-4 cell-2 reinstall tray in mer,False,Cluster#45
3385,"3,192,764",MDC1-10564,COLO-4 Cell-3 Remove inaccessible tray in MER,colo-4 cell-3 remove inaccessible tray in mer,False,Cluster#44


In [38]:
data.head()

,id,activity_id,name,lower_case_names,is_one_word_name,cluster
0,"3,191,380",MDC1-UP#13,MDC1 Master Update #13- DD 8.31.2021Submitted,mdc1 master update #13- dd 8.31.2021submitted,False,NaN
1,"3,191,381",MDC1-UP#13.1,Executive Summary / Milestones,executive summary / milestones,False,NaN
2,"3,192,817",MDC1-UP#13.5,Construction,construction,True,Cluster#1
3,"3,191,388",MDC1-UP#13.2,Construction Summary,construction summary,False,Cluster#1
4,"3,191,673",MDC1-UP#13.4,Preconstruction,preconstruction,True,Cluster#2


In [39]:
data.cluster.isna().count()

3436

In [40]:
3436 - 55

3381

In [41]:
data.to_csv("naive_clusters.csv")